In [1]:
%%time
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

CPU times: user 1.58 s, sys: 247 ms, total: 1.82 s
Wall time: 1.67 s


In [2]:
tf.enable_v2_behavior()

In [3]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [4]:
(ds_train, ds_test), ds_info = tfds.load( 'mnist', split=['train', 'test'], shuffle_files=True, as_supervised=True, with_info=True, )


2021-12-19 19:07:13.938432: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /Users/matalcross_imac/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
Metal device set to: Apple M1


2021-12-19 19:07:21.199270: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-19 19:07:21.199385: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label

In [6]:
batch_size = 128

In [7]:
ds_train = ds_train.map( normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
ds_test = ds_test.map( normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [10]:
model = tf.keras.models.Sequential([ 
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'), 
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)), 
    # tf.keras.layers.Dropout(0.25), 
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(128, activation='relu'), 
    # tf.keras.layers.Dropout(0.5), 
    tf.keras.layers.Dense(10, activation='softmax') ])

In [11]:
model.compile( loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001), metrics=['accuracy'], )

In [12]:
model.fit( ds_train, epochs=12, validation_data=ds_test, )

2021-12-19 19:12:59.673872: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-19 19:12:59.673924: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-12-19 19:12:59.680764: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-19 19:12:59.778013: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-19 19:12:59.787813: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-19 19:13:00.306117: I tensorflow/core/grappler/o

Train on 469 steps, validate on 79 steps
Epoch 1/12
469/469 [==============================] - ETA: 0s - batch: 234.0000 - size: 1.0000 - loss: 0.1555 - accuracy: 0.9538

/opt/homebrew/Caskroom/miniforge/base/envs/p39t27/lib/python3.9/site-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2021-12-19 19:13:12.955133: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - 13s 20ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.1555 - accuracy: 0.9538 - val_loss: 0.0473 - val_accuracy: 0.9853
Epoch 2/12
469/469 [==============================] - 10s 20ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0439 - accuracy: 0.9867 - val_loss: 0.0365 - val_accuracy: 0.9872
Epoch 3/12
469/469 [==============================] - 10s 19ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0279 - accuracy: 0.9916 - val_loss: 0.0426 - val_accuracy: 0.9856
Epoch 4/12
469/469 [==============================] - 10s 19ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0198 - accuracy: 0.9937 - val_loss: 0.0326 - val_accuracy: 0.9897
Epoch 5/12
469/469 [==============================] - 10s 20ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0125 - accuracy: 0.9960 - val_loss: 0.0407 - val_accuracy: 0.9876
Epoch 6/12
469/469 [==============================] - 10s 20ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0101 - accuracy